# Exploration des Données pour la Génération de Lunettes 3D

Ce notebook explore les données d'images de lunettes et leurs caractéristiques pour l'adaptation du modèle Hunyuan3D 2.0.

In [ ]:
import os
import sys
import json
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import glob
import torch
import torchvision.transforms as transforms
from tqdm.notebook import tqdm

# Ajout du répertoire parent au path pour importer les modules personnalisés
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))
from utils.dataset import GlassesDataset

## 1. Configuration

In [ ]:
# Répertoire des données
DATA_DIR = "../data/train"

# Vérification de l'existence du répertoire
if not os.path.exists(DATA_DIR):
    print(f"Le répertoire {DATA_DIR} n'existe pas. Créez-le et ajoutez-y des images de lunettes.")
else:
    print(f"Répertoire des données: {DATA_DIR}")

## 2. Exploration des Images

In [ ]:
# Recherche des images dans le répertoire
image_paths = []
for ext in ["*.jpg", "*.jpeg", "*.png"]:
    image_paths.extend(glob.glob(os.path.join(DATA_DIR, "images", ext)))

print(f"Nombre d'images trouvées: {len(image_paths)}")

# Affichage de quelques images
if len(image_paths) > 0:
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    for i, img_path in enumerate(image_paths[:6]):
        img = Image.open(img_path)
        axes[i].imshow(img)
        axes[i].set_title(os.path.basename(img_path))
        axes[i].axis("off")
    
    plt.tight_layout()
    plt.show()

## 3. Analyse des Caractéristiques des Lunettes

In [ ]:
def extract_glasses_features(image_path):
    """Extraction des caractéristiques des lunettes à partir d'une image"""
    # Chargement de l'image
    image = Image.open(image_path).convert("RGB")
    image_np = np.array(image)
    
    # Conversion en niveaux de gris
    gray = cv2.cvtColor(image_np, cv2.COLOR_RGB2GRAY)
    
    # Flou gaussien pour réduire le bruit
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Détection des contours
    edges = cv2.Canny(blurred, 50, 150)
    
    # Recherche des contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Caractéristiques
    features = {}
    
    if contours:
        # Trouver le plus grand contour (supposé être les lunettes)
        largest_contour = max(contours, key=cv2.contourArea)
        
        # Rectangle englobant
        x, y, w, h = cv2.boundingRect(largest_contour)
        features["bounding_box"] = (x, y, w, h)
        
        # Centre
        M = cv2.moments(largest_contour)
        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
        else:
            cx, cy = x + w // 2, y + h // 2
        features["center"] = (cx, cy)
        
        # Aire
        features["area"] = cv2.contourArea(largest_contour)
        
        # Périmètre
        features["perimeter"] = cv2.arcLength(largest_contour, True)
        
        # Orientation (si possible)
        if len(largest_contour) >= 5:
            (x_ellipse, y_ellipse), (MA, ma), angle = cv2.fitEllipse(largest_contour)
            features["orientation"] = angle
            features["aspect_ratio"] = MA / ma if ma > 0 else 0
        
        # Visualisation
        vis_image = image_np.copy()
        cv2.drawContours(vis_image, [largest_contour], 0, (0, 255, 0), 2)
        cv2.rectangle(vis_image, (x, y), (x + w, y + h), (255, 0, 0), 2)
        cv2.circle(vis_image, (cx, cy), 5, (0, 0, 255), -1)
        
        features["visualization"] = vis_image
    
    return features

# Analyse de quelques images
if len(image_paths) > 0:
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    for i, img_path in enumerate(image_paths[:6]):
        features = extract_glasses_features(img_path)
        
        if "visualization" in features:
            axes[i].imshow(features["visualization"])
            title = f"BB: {features['bounding_box']}\nArea: {features['area']:.0f}"
            if "orientation" in features:
                title += f"\nAngle: {features['orientation']:.1f}°"
            axes[i].set_title(title)
        else:
            img = Image.open(img_path)
            axes[i].imshow(img)
            axes[i].set_title("Pas de contour détecté")
        
        axes[i].axis("off")
    
    plt.tight_layout()
    plt.show()

## 4. Utilisation du Dataset

In [ ]:
# Création du dataset
dataset = GlassesDataset(DATA_DIR)
print(f"Taille du dataset: {len(dataset)}")

# Affichage de quelques exemples
if len(dataset) > 0:
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    for i in range(min(6, len(dataset))):
        sample = dataset[i]
        img = sample["image"]
        
        # Conversion du tensor en image
        img = img.numpy().transpose(1, 2, 0)
        img = (img * 0.5 + 0.5).clip(0, 1)  # Dénormalisation
        
        axes[i].imshow(img)
        axes[i].set_title(sample["image_name"])
        axes[i].axis("off")
    
    plt.tight_layout()
    plt.show()

## 5. Analyse des Métadonnées

In [ ]:
# Chargement des métadonnées si elles existent
metadata_path = os.path.join(DATA_DIR, "metadata.json")
if os.path.exists(metadata_path):
    with open(metadata_path, "r") as f:
        metadata = json.load(f)
    
    print(f"Nombre d'entrées dans les métadonnées: {len(metadata)}")
    
    # Affichage des clés pour une entrée
    if len(metadata) > 0:
        first_key = list(metadata.keys())[0]
        print(f"\nClés pour {first_key}:")
        for key, value in metadata[first_key].items():
            print(f"  {key}: {value}")
    
    # Analyse statistique
    if len(metadata) > 0:
        # Extraction des caractéristiques
        widths = [item["width"] for item in metadata.values() if "width" in item]
        heights = [item["height"] for item in metadata.values() if "height" in item]
        areas = [item["area"] for item in metadata.values() if "area" in item]
        angles = [item["angle"] for item in metadata.values() if "angle" in item]
        
        # Statistiques
        print("\nStatistiques:")
        print(f"  Largeur moyenne: {np.mean(widths):.1f} ± {np.std(widths):.1f}")
        print(f"  Hauteur moyenne: {np.mean(heights):.1f} ± {np.std(heights):.1f}")
        print(f"  Aire moyenne: {np.mean(areas):.1f} ± {np.std(areas):.1f}")
        print(f"  Angle moyen: {np.mean(angles):.1f}° ± {np.std(angles):.1f}°")
        
        # Visualisation
        fig, axes = plt.subplots(2, 2, figsize=(12, 10))
        
        axes[0, 0].hist(widths, bins=20)
        axes[0, 0].set_title("Distribution des largeurs")
        axes[0, 0].set_xlabel("Largeur (pixels)")
        
        axes[0, 1].hist(heights, bins=20)
        axes[0, 1].set_title("Distribution des hauteurs")
        axes[0, 1].set_xlabel("Hauteur (pixels)")
        
        axes[1, 0].hist(areas, bins=20)
        axes[1, 0].set_title("Distribution des aires")
        axes[1, 0].set_xlabel("Aire (pixels²)")
        
        axes[1, 1].hist(angles, bins=20)
        axes[1, 1].set_title("Distribution des angles")
        axes[1, 1].set_xlabel("Angle (degrés)")
        
        plt.tight_layout()
        plt.show()
else:
    print(f"Pas de métadonnées trouvées à {metadata_path}")

## 6. Conclusion

Cette analyse des données nous a permis de mieux comprendre les caractéristiques des images de lunettes dans notre dataset. Ces informations seront utiles pour adapter le modèle Hunyuan3D 2.0 à la génération de lunettes 3D.